<a href="https://colab.research.google.com/github/n-vit/YaP_Projects/blob/main/identifying_toxic_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Отсев токсичных комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [ ]:
import pandas as pd 
import numpy as np 

from sklearn.pipeline import Pipeline
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier 

from sklearn.metrics import f1_score

import time
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

import plotly.graph_objects as go

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
try: # if the project is opened in Collab use a 'try' block, if it opened in simulator Yandex, or local Jupiter, use an 'except' block
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

except:
    df = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
df.head(3)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


In [ ]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.iloc[2][0]

'hey man i m really not trying to edit war it s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info'

Итак, у нас есть фрейм из 159 тыс. строк, без пропусков и дубликатов, содержащий текстовые твиты и бинарный таргет токсичности. 

Твиты содержат живой англоязычный текст, со знаками препинания, различными регистрами букв, пробелами и ошибками. Токсичных твитов немногим меньше 10% от фрейма


## Подготовка

### Очистка
оставим в тексте только латиницу, без знаков препинания и других символов

<div class="alert alert-block alert-warning">
<b>Изменения:</b> 
    
Добавил в функцию очистки лемматизацию WordNetLemmatizer.

    
</div>

In [ ]:

wnl = WordNetLemmatizer()
def clear_text(text):
    clean_text = re.sub(r'[^a-zA-Z]',' ',text)
    clean_text = " ".join(clean_text.split()).lower()
    clean_text = nltk.word_tokenize(clean_text)
    clean_text =  ' '.join([wnl.lemmatize(w) for w in clean_text])
    return clean_text

In [ ]:
df['text'] = df['text'].map(lambda x: clear_text(x))

In [ ]:
df.iloc[1][0]

'd aww he match this background colour i m seemingly stuck with thanks talk january utc'

Выделим таргет и разобьем выборки на учебную и тестовую (70/30)

In [ ]:
features = df.drop('toxic', axis=1)
target = df['toxic']

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size = 0.3, random_state=12345)

## Обучение 

### Логистическая регрессия
подберем гиперпараметры и обучим модель:

In [ ]:

%%time
'''
lr_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), 
                              min_df=3, max_df=0.9, 
                              use_idf=1, smooth_idf=1, 
                              sublinear_tf=1, stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=130682))])

params = {'clf__C': [0.1, 1, 10, 100],
          'clf__class_weight': ['balanced', None]}

lr_grid = GridSearchCV(estimator=lr_pipe, 
                       param_grid=params, 
                       cv=3, scoring='f1', 
                       n_jobs=-1, refit=False)
lr_grid.fit(features_train['text'], target_train)
lr_best_paramms = lr_grid.best_params_

print(lr_best_paramms)
print(lr_grid.best_score_)
'''

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.2 µs


"\nlr_pipe = Pipeline([\n    ('tfidf', TfidfVectorizer(ngram_range=(1,3), \n                              min_df=3, max_df=0.9, \n                              use_idf=1, smooth_idf=1, \n                              sublinear_tf=1, stop_words=stopwords)),\n    ('clf', LogisticRegression(random_state=130682))])\n\nparams = {'clf__C': [0.1, 1, 10, 100],\n          'clf__class_weight': ['balanced', None]}\n\nlr_grid = GridSearchCV(estimator=lr_pipe, \n                       param_grid=params, \n                       cv=3, scoring='f1', \n                       n_jobs=-1, refit=False)\nlr_grid.fit(features_train['text'], target_train)\nlr_best_paramms = lr_grid.best_params_\n\nprint(lr_best_paramms)\nprint(lr_grid.best_score_)\n"

Лучшие параметры на кросс-валидации дают приемлемый результат (0,776) на обучающей выборке. 

Результаты обучения: (*код закомментирован для экономии времени*):

{'clf__C': 10, 'clf__class_weight': 'balanced'}
0.7758575228347439
CPU times: user 7.13 s, sys: 1.13 s, total: 8.27 s
Wall time: 8min 54s

### Градиентный бустинг LightGBM

In [ ]:
'''
%%time

lgb_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), 
                              min_df=3, max_df=0.9, 
                              use_idf=1, smooth_idf=1, 
                              sublinear_tf=1, stop_words=stopwords)),
    ('clf', LGBMClassifier(random_state=130682))])

params = {
  'clf__n_estimators': [200],
  'clf__learning_rate': [0.15, 0.25],
  'clf__max_depth': [8, 10, -1]}

lgb_grid = GridSearchCV(estimator=lgb_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
lgb_grid.fit(features_train['text'], target_train)
lgb_best_params = lgb_grid.best_params_

print(lgb_best_params)
print(lgb_grid.best_score_)
'''

"\n%%time\n\nlgb_pipe = Pipeline([\n    ('tfidf', TfidfVectorizer(ngram_range=(1,3), \n                              min_df=3, max_df=0.9, \n                              use_idf=1, smooth_idf=1, \n                              sublinear_tf=1, stop_words=stopwords)),\n    ('clf', LGBMClassifier(random_state=130682))])\n\nparams = {\n  'clf__n_estimators': [200],\n  'clf__learning_rate': [0.15, 0.25],\n  'clf__max_depth': [8, 10, -1]}\n\nlgb_grid = GridSearchCV(estimator=lgb_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)\nlgb_grid.fit(features_train['text'], target_train)\nlgb_best_params = lgb_grid.best_params_\n\nprint(lgb_best_params)\nprint(lgb_grid.best_score_)\n"

Бустинг на обучающей выборке получился чуть хуже (0,768) а училась модель почти в четыре раза дольше. 

Результаты обучения: (*код закомментирован для экономии времени*):


{'clf__learning_rate': 0.25, 'clf__max_depth': -1, 'clf__n_estimators': 200}
0.7695633634325536
CPU times: user 13.8 s, sys: 2.36 s, total: 16.1 s
Wall time: 31min 49s

## Тестирование
векторизируем твиты с помощью TfidfVectorizer, 

In [ ]:
vectorize = TfidfVectorizer(ngram_range=(1,3),
               min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)

In [ ]:
features_train = vectorize.fit_transform(features_train['text'])
features_test = vectorize.transform(features_test['text'])

обучим модели на лучших гиперпараметрах

In [ ]:
%%time
lr_m = LogisticRegression(C=10, 
                          class_weight='balanced', 
                          random_state=130682)
lr_m.fit(features_train, target_train)

CPU times: user 23.6 s, sys: 36.7 s, total: 1min
Wall time: 1min


LogisticRegression(C=10, class_weight='balanced', random_state=130682)

In [ ]:
%%time
lgb_m = LGBMClassifier(learning_rate=0.25, 
                       max_depth=-1, 
                       n_estimators=200, 
                       random_state = 130682)
lgb_m.fit(features_train, target_train)

CPU times: user 16min 30s, sys: 0 ns, total: 16min 30s
Wall time: 16min 32s


LGBMClassifier(learning_rate=0.25, n_estimators=200, random_state=130682)

Посчитаем F1 для логистической регрессии и градиентного бустинга LGBM

In [ ]:
def scoring(fitted_model):
    test_pred = fitted_model.predict(features_test)
    test_f1 = f1_score(target_test, test_pred)
    
    return test_f1


In [ ]:
%%time
print('F1 градиентного бустинга на тестовой выборке: {:.3f}'.format(scoring(lgb_m)))
print()

F1 градиентного бустинга на тестовой выборке: 0.776

CPU times: user 7.11 s, sys: 0 ns, total: 7.11 s
Wall time: 7.09 s


In [ ]:
%%time
print('F1 логистической регрессии на тестовой выборке: {:.3f}'.format(scoring(lr_m)))
print()

F1 логистической регрессии на тестовой выборке: 0.782

CPU times: user 22 ms, sys: 0 ns, total: 22 ms
Wall time: 73 ms


## Выводы

- фрейм с разметкой комментариев позволяет обучить модели и определять настроение (токсичность) комментариев с заданным качеством (F1 не менее 0,75)
- обучены и протестированы модели на основе логистической регрессии и градиентного бустинга lightGBM. 

Результаты F1 моделей сопоставимы и отличаются всего на пять тысячных, но бустинг учится и работает значительно дольше. 

**Обученную модель логистической регрессии  с результатом предсказания F1=0.784 и временем предсказания 43 милисекунды можно рекомендовать к дальнейшей работе**

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны